In [ ]:
!git clone https://github.com/Cornell-RelaxML/Hyperdimensional-Computing

Cloning into 'Hyperdimensional-Computing'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 30 (delta 0), reused 1 (delta 0), pack-reused 27 (from 1)
Receiving objects: 100% (30/30), 48.17 MiB | 17.81 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
%cd /content/Hyperdimensional-Computing

/content/Hyperdimensional-Computing


# Importing librairies and Functions' definition

In [ ]:
import torch
import numpy as np
import time
import os
from utils import prepare_data, encode_and_save
from model import BModel, GModel
import argparse
import torch.nn as nn
import torch.optim as optim

In [ ]:
def test(MODEL, loader, criterion, device, model_='rff-hdc'):
    MODEL.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            if model_ == 'rff-hdc':
                outputs = MODEL(2 * inputs - 1)
            else:
                outputs = MODEL(inputs)
            test_loss += criterion(outputs, labels)
            preds = outputs.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += preds.eq(labels.view_as(preds)).sum().item()
    test_loss /= len(loader.dataset)
    print('\nTesting Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(loader.dataset),
        100. * correct / len(loader.dataset)))

In [ ]:
def train(args):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # criterion = torch.nn.NLLLoss()
    criterion = torch.nn.CrossEntropyLoss()
    channels = 3 if args.dataset == 'cifar' else 1
    if args.dataset == 'isolet':
        classes = 26
    elif args.dataset == 'ucihar':
        classes = 6
    else:
        classes = 10
    if 'hdc' in args.model:
        model = BModel(in_dim=channels * args.dim, classes=classes).to(device)
    elif 'percep' in args.model:
        model = nn.Sequential(nn.Linear(10000, classes)).to(device)
    else:
        model = GModel(args.gorder, in_dim=channels * args.dim, classes=classes).to(device)

    trainloader, testloader = prepare_data(args)
    #     optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)#, momentum=0.9, weight_decay=1.0e-5)
    #     optimizer = torch.optim.Adadelta(model.parameters(), lr=args.lr)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

    for epoch in range(args.epoch):
        print("Epoch:", epoch + 1)
        model.train()
        start_time = time.time()
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            if args.model == 'rff-hdc':
                outputs = model(2 * inputs - 1)
            else:
                outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            _, batch_predicted = torch.max(outputs.data, 1)
            batch_accu = 100.0 * (batch_predicted == labels).sum().item() / labels.size(0)
            if i % 50 == 49:
                print(i, "{0:.4f}".format(loss.item()), batch_accu)
        print('Start testing on test set')
        test(model, testloader, criterion, device, model_=args.model)
        print("--- %s seconds ---" % (time.time() - start_time))


# Perceptron RFF on the different data sets

## ISOLET

In [ ]:
#isolet epoch 1 percep
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'percep-rff'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 6238 1559
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.5343, -1.1505, -0.8872, -0.6746, -0.4888, -0.3187, -0.1573,  0.0000,
         0.1573,  0.3187,  0.4888,  0.6746,  0.8872,  1.1505,  1.5343])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1


/content/Hyperdimensional-Computing/utils.py:116: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_hd = torch.load(f'{args.data_dir}/train_hd.pt')
/content/Hyperdimension

49 995858.8750 6.25
99 526116.8125 6.25
149 809939.2500 6.25
199 913873.3750 0.0
249 697573.6250 0.0
299 453530.5000 0.0
349 403353.8125 0.0
Start testing on test set

Testing Average loss: 48020.4922, Accuracy: 60/1559 (3.85%)

--- 2.5029330253601074 seconds ---


In [ ]:
#isolet Percep epoch 10
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'percep-rff'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 868007.8750 6.25
99 827970.8125 0.0
149 425642.5000 0.0
199 330255.8125 12.5
249 561979.3750 0.0
299 686604.0000 0.0
349 523490.1250 0.0
Start testing on test set

Testing Average loss: 36815.3633, Accuracy: 60/1559 (3.85%)

--- 1.6314761638641357 seconds ---
Epoch: 2
49 840174.5000 6.25
99 665619.8750 0.0
149 400420.2500 6.25
199 423796.6250 6.25
249 832145.0000 6.25
299 762263.2500 0.0
349 597408.7500 0.0
Start testing on test set

Testing Average loss: 41860.8242, Accuracy: 60/1559 (3.85%)

--- 1.6185083389282227 seconds ---
Epoch: 3
49 756467.9375 0.0
99 657288.5625 0.0
149 369311.8438 0.0
199 405602.5938 6.25
249 923299.7500 0.0
299 692592.8125 12.5
349 468610.5625 0.0
Start testing on test set

Testing Average loss: 37269.1211, Accu

## UCIHAR


In [ ]:
#UCIHAR Percep epoch 1
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'percep-rff'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 7352 2947
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.5343, -1.1505, -0.8872, -0.6746, -0.4888, -0.3187, -0.1573,  0.0000,
         0.1573,  0.3187,  0.4888,  0.6746,  0.8872,  1.1505,  1.5343])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 246569.7500 25.0
99 74794.5000 6.25
149 286504.1562 12.5
199 158004.5469 25.0
249 195212.6250 12.5
299 85235.6719 18.75
349 95910.5625 18.75
399 135563.7812 25.0
449 180684.5938 

In [ ]:
#UCIHAR Percep epoch 10
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'percep-rff'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 147516.7031 0.0
99 232809.3438 25.0
149 165723.8281 37.5
199 282995.5625 6.25
249 166385.8594 18.75
299 301754.4062 18.75
349 137537.0156 18.75
399 220966.9688 12.5
449 168522.3750 37.5
Start testing on test set

Testing Average loss: 17324.5273, Accuracy: 496/2947 (16.83%)

--- 2.063068389892578 seconds ---
Epoch: 2
49 47441.6875 31.25
99 224030.3750 6.25
149 179364.0625 12.5
199 158497.4062 6.25
249 72886.6641 12.5
299 66122.3203 12.5
349 265680.1250 31.25
399 237218.3750 18.75
449 227155.3750 6.25
Start testing on test set

Testing Average loss: 5479.3911, Accuracy: 532/2947 (18.05%)

--- 2.035463333129883 seconds ---
Epoch: 3
49 170961.6562 25.0
99 160176.5000 6.25
149 218127.7188 0.0
199 259056.4688 6.25
249 287386.6250 12.5
299 1991

## MNIST

In [ ]:
#mnist Percep epoch 1
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'percep-rff'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 17.8MB/s]


Extracting ./dataset/MNIST/raw/train-images-idx3-ubyte.gz to ./dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 486kB/s]


Extracting ./dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ./dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.49MB/s]


Extracting ./dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to ./dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.55MB/s]


Extracting ./dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./dataset/MNIST/raw

# of channels of data 1
# of training samples and test samples 60000 10000
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.5343, -1.1505, -0.8872, -0.6746, -0.4888, -0.3187, -0.1573,  0.0000,
         0.1573,  0.3187,  0.4888,  0.6746,  0.8872,  1.1505,  1.5343])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
12800 images encoded. Total time elapse = 23.742467164993286
25600 images encoded. Total time elapse = 46.900729179382324
38400 images encoded. Total time elapse = 69.94405055046082
51200 images encoded. Total time elapse = 93.06130862236023
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of

In [ ]:
#mnist Percep epoch 10
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'percep-rff'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 564582.4375 0.0
99 267302.0312 12.5
149 321624.1250 25.0
199 229842.5938 0.0
249 183192.1250 6.25
299 373801.5000 25.0
349 458999.3125 12.5
399 479386.8125 18.75
449 318339.2812 12.5
499 471671.3125 6.25
549 266498.3750 18.75
599 261286.4844 0.0
649 438965.8438 12.5
699 262413.3750 18.75
749 370561.4688 6.25
799 346419.2812 12.5
849 274350.8125 6.25
899 700092.1875 12.5
949 472051.5000 12.5
999 202532.4219 25.0
1049 455981.1875 12.5
1099 394978.0938 12.5
1149 228486.7500 12.5
1199 178865.1406 6.25
1249 228810.0625 25.0
1299 462197.6250 12.5
1349 560815.5625 0.0
1399 698394.3125 0.0
1449 502243.6875 6.25
1499 436863.6875 6.25
1549 283534.0000 0.0
1599 447018.5000 6.25
1649 358891.0000 6.25
1699 384533.5000 12.5
1749 379311.8125 12.5
1799

## FMNIST

In [ ]:
#fmnist Percep epoch 1
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'percep-rff'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save


100%|██████████| 26.4M/26.4M [00:01<00:00, 13.6MB/s]


Extracting ./dataset/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./dataset/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 200kB/s]


Extracting ./dataset/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./dataset/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.75MB/s]


Extracting ./dataset/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./dataset/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 6.42MB/s]


Extracting ./dataset/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./dataset/FashionMNIST/raw

# of channels of data 1
# of training samples and test samples 60000 10000
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.5343, -1.1505, -0.8872, -0.6746, -0.4888, -0.3187, -0.1573,  0.0000,
         0.1573,  0.3187,  0.4888,  0.6746,  0.8872,  1.1505,  1.5343])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
12800 images encoded. Total time elapse = 27.41977572441101
25600 images encoded. Total time elapse = 54.7140851020813
38400 images encoded. Total time elapse = 81.93553471565247
51200 images encoded. Total time elapse = 109.11835789680481
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test dat

In [ ]:
#fmnist Percep epoch 10
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'percep-rff'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 341323.2188 6.25
99 613184.3125 0.0
149 491808.6250 12.5
199 396343.5000 0.0
249 143109.0938 12.5
299 241291.2344 12.5
349 550162.6875 0.0
399 410321.6875 6.25
449 178241.2031 18.75
499 419919.7500 6.25
549 292132.3750 18.75
599 501967.1250 6.25
649 439897.4375 6.25
699 357114.0000 6.25
749 403396.8125 6.25
799 205117.6719 18.75
849 623652.8125 6.25
899 283737.7500 6.25
949 384775.9375 6.25
999 288441.0625 0.0
1049 232387.1406 6.25
1099 208408.4375 0.0
1149 192921.5312 6.25
1199 477143.2188 12.5
1249 288184.8750 6.25
1299 264831.6875 6.25
1349 559181.0625 6.25
1399 232921.2656 25.0
1449 211989.5156 25.0
1499 474718.0312 0.0
1549 460942.6875 12.5
1599 402281.2812 12.5
1649 689392.7500 0.0
1699 221355.3906 6.25
1749 409987.2500 12.5
1799 

# Linear HDC

## ISOLET

In [ ]:
#isolet epoch 1 linear-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'linear-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 6238 1559
Encoding to binary HDC with linear hamming distance.
generating linear item memory...
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
start encoding data here
1000 images encoded
2000 images encoded
3000 images encoded
4000 images encoded
5000 images encoded
6000 images encoded
finish encoding data here
Encoding test data...
start encoding data here
1000 images encoded
finish encoding data here
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 3.6680 6.25
99 3.6751 12.5
149 3.4996 0.0
199 3.6306 18.75
249 3.6036 6.25
299 3.7174 6.25
349 3.4921 6.25
Start testing o

In [ ]:
#isolet epoch 10 linear-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'linear-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 3.6680 6.25
99 3.6751 12.5
149 3.4996 0.0
199 3.6306 18.75
249 3.6036 6.25
299 3.7174 6.25
349 3.4921 6.25
Start testing on test set

Testing Average loss: 0.2350, Accuracy: 56/1559 (3.59%)

--- 3.440319299697876 seconds ---
Epoch: 2
49 0.3289 100.0
99 0.3025 100.0
149 0.5690 100.0
199 0.6325 100.0
249 0.7024 100.0
299 0.9241 100.0
349 0.8942 100.0
Start testing on test set

Testing Average loss: 0.2346, Accuracy: 54/1559 (3.46%)

--- 2.488813877105713 seconds ---
Epoch: 3
49 0.3354 100.0
99 0.3432 100.0
149 0.5210 100.0
199 0.6594 100.0
249 0.6662 100.0
299 0.6389 100.0
349 0.9188 87.5
Start testing on test set

Testing Average loss: 0.2359, Accuracy: 50/1559 (3.21%)

## UCIHAR

In [ ]:
#ucihar epoch 1 linear-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'linear-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 7352 2947
Encoding to binary HDC with linear hamming distance.
generating linear item memory...
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
start encoding data here
1000 images encoded
2000 images encoded
3000 images encoded
4000 images encoded
5000 images encoded
6000 images encoded
7000 images encoded
finish encoding data here
Encoding test data...
start encoding data here
1000 images encoded
2000 images encoded
finish encoding data here
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 2.1905 18.75
99 2.3607 18.75
149 2.3786 6.25
199 2.0715 25.0
249 1.7966 25.0
299 2

In [ ]:
#ucihar epoch 10 linear-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'linear-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 2.1905 18.75
99 2.3607 18.75
149 2.3786 6.25
199 2.0715 25.0
249 1.7966 25.0
299 2.1223 12.5
349 2.1425 18.75
399 2.0151 25.0
449 2.4769 18.75
Start testing on test set

Testing Average loss: 0.1382, Accuracy: 469/2947 (15.91%)

--- 3.045701026916504 seconds ---
Epoch: 2
49 0.6541 87.5
99 0.7335 75.0
149 1.2067 50.0
199 0.9138 62.5
249 1.1908 62.5
299 1.1308 50.0
349 1.2650 37.5
399 1.0606 50.0
449 1.0830 56.25
Start testing on test set

Testing Average loss: 0.1383, Accuracy: 487/2947 (16.53%)

--- 2.320056915283203 seconds ---
Epoch: 3
49 0.7935 75.0
99 0.5335 93.75
149 0.8218 68.75
199 0.7779 75.0
249 0.9077 62.5
299 0.9645 62.5
349 1.1450 68.75
399 1.0969 68.75
44

## MNIST

In [ ]:
#mnist epoch 1 linear-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'linear-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 60000 10000
Encoding to binary HDC with linear hamming distance.
generating linear item memory...
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
start encoding data here
1000 images encoded
2000 images encoded
3000 images encoded
4000 images encoded
5000 images encoded
6000 images encoded
7000 images encoded
8000 images encoded
9000 images encoded
10000 images encoded
11000 images encoded
12000 images encoded
13000 images encoded
14000 images encoded
15000 images encoded
16000 images encoded
17000 images encoded
18000 images encoded
19000 images encoded
20000 images encoded
21000 images encoded
22000 images encoded
23000 images encoded
24000 images encoded
25000 images encoded
26000 images encoded
27000 images encoded
28000 images encoded
29000 images encoded
30

In [ ]:
#mnist epoch 10 linear-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'linear-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 3.0296 0.0
99 2.6802 12.5
149 2.9251 6.25
199 2.5587 6.25
249 2.5108 25.0
299 3.0860 6.25
349 2.6826 12.5
399 2.6276 6.25
449 2.6436 25.0
499 2.7960 6.25
549 2.9707 6.25
599 2.4753 25.0
649 2.5953 18.75
699 2.7661 6.25
749 3.0150 6.25
799 2.6468 0.0
849 2.5650 6.25
899 3.0149 6.25
949 2.7726 12.5
999 2.6841 18.75
1049 2.7173 6.25
1099 2.6282 6.25
1149 2.5664 12.5
1199 2.7995 0.0
1249 2.7478 18.75
1299 2.6889 6.25
1349 2.6242 12.5
1399 2.4804 18.75
1449 2.4388 6.25
1499 2.8117 12.5
1549 2.6075 12.5
1599 2.7489 0.0
1649 2.7472 12.5
1699 2.7073 12.5
1749 2.5681 6.25
1799 2.7572 12.5
1849 2.8757 12.5
1899 2.9989 18.75
1949 2.9197 0.0
1999 2.6215 12.5
2049 3.0145 6.25
20

## FMNIST

In [ ]:
#fminst epoch 1 linear-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'linear-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 60000 10000
Encoding to binary HDC with linear hamming distance.
generating linear item memory...
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
start encoding data here
1000 images encoded
2000 images encoded
3000 images encoded
4000 images encoded
5000 images encoded
6000 images encoded
7000 images encoded
8000 images encoded
9000 images encoded
10000 images encoded
11000 images encoded
12000 images encoded
13000 images encoded
14000 images encoded
15000 images encoded
16000 images encoded
17000 images encoded
18000 images encoded
19000 images encoded
20000 images encoded
21000 images encoded
22000 images encoded
23000 images encoded
24000 images encoded
25000 images encoded
26000 images encoded
27000 images encoded
28000 images encoded
29000 images encoded
30

In [ ]:
#fminst epoch 10 linear-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'linear-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 2.5683 18.75
99 2.8018 18.75
149 2.5834 18.75
199 2.7154 0.0
249 2.4274 12.5
299 2.2016 25.0
349 2.6011 12.5
399 2.4826 6.25
449 2.8634 0.0
499 2.3867 18.75
549 2.8498 6.25
599 2.8376 0.0
649 3.1383 0.0
699 2.7834 12.5
749 2.5576 6.25
799 2.5611 12.5
849 2.7648 18.75
899 2.6754 12.5
949 2.6073 31.25
999 3.1782 0.0
1049 2.5408 12.5
1099 2.2742 31.25
1149 3.2473 0.0
1199 3.0077 12.5
1249 2.7598 12.5
1299 2.7092 0.0
1349 2.6402 12.5
1399 2.8426 0.0
1449 2.1192 50.0
1499 2.9140 12.5
1549 2.9010 6.25
1599 2.5573 18.75
1649 2.9761 0.0
1699 3.0149 6.25
1749 3.1668 6.25
1799 2.5949 6.25
1849 2.7213 12.5
1899 3.3172 0.0
1949 2.5461 18.75
1999 2.6413 12.5
2049 2.9021 18.75
20

# RFF HDC

## ISOLET

In [ ]:
#isolet epoch 1 rff-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 6238 1559
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([0.])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 0.8162 87.5
99 0.5102 87.5
149 0.4082 87.5
199 0.1630 93.75
249 0.8207 75.0
299 0.4206 81.25
349 0.1552 100.0
Start testing on test set

Testing Average loss: 0.0205, Accuracy: 1411/1559 (90.51%)

--- 3.6395585536956787 seconds ---


In [ ]:
#isolet epoch 10 rff-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 0.6727 75.0
99 0.5162 75.0
149 0.2010 93.75
199 0.8416 81.25
249 0.5621 87.5
299 0.6517 75.0
349 0.2199 93.75
Start testing on test set

Testing Average loss: 0.0235, Accuracy: 1382/1559 (88.65%)

--- 1.8294272422790527 seconds ---
Epoch: 2
49 0.0897 100.0
99 0.0332 100.0
149 0.0098 100.0
199 0.0389 100.0
249 0.0323 100.0
299 0.0946 93.75
349 0.0373 100.0
Start testing on test set

Testing Average loss: 0.0184, Accuracy: 1414/1559 (90.70%)

--- 1.846909523010254 seconds ---
Epoch: 3
49 0.0072 100.0
99 0.0043 100.0
149 0.0050 100.0
199 0.0144 100.0
249 0.0076 100.0
299 0.0064 100.0
349 0.0131 100.0
Start testing on test set

Testing Average loss: 0.0177, Accuracy: 1418

## UCIHAR

In [ ]:
#ucihar epoch 1 rff-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 7352 2947
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([0.])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 0.9588 75.0
99 0.0221 100.0
149 0.5313 87.5
199 0.0612 100.0
249 0.1879 93.75
299 0.0794 100.0
349 0.4489 81.25
399 0.2374 87.5
449 1.0105 87.5
Start testing on test set

Testing Average loss: 0.0577, Accuracy: 2316/2947 (78.59%)

--- 2.27528357

In [ ]:
#ucihar epoch 10 rff-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 0.8315 75.0
99 0.0903 100.0
149 0.2130 93.75
199 0.3415 87.5
249 0.2836 93.75
299 0.0112 100.0
349 0.1453 87.5
399 0.5739 81.25
449 0.0114 100.0
Start testing on test set

Testing Average loss: 0.0112, Accuracy: 2753/2947 (93.42%)

--- 3.0124118328094482 seconds ---
Epoch: 2
49 0.0604 93.75
99 0.0328 100.0
149 0.0081 100.0
199 0.0037 100.0
249 0.0503 100.0
299 0.0153 100.0
349 0.6410 81.25
399 0.0426 100.0
449 0.0162 100.0
Start testing on test set

Testing Average loss: 0.0156, Accuracy: 2715/2947 (92.13%)

--- 2.402364730834961 seconds ---
Epoch: 3
49 0.0040 100.0
99 0.0029 100.0
149 0.0114 100.0
199 0.0145 100.0
249 0.0102 100.0
299 0.0085 100.0
349 0.1522 93.75
39

## MNIST

In [ ]:
#mnist epoch 1 rff-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 60000 10000
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([0.])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
12800 images encoded. Total time elapse = 22.82528042793274
25600 images encoded. Total time elapse = 45.816142320632935
38400 images encoded. Total time elapse = 68.91876697540283
51200 images encoded. Total time elapse = 91.96357250213623
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 

In [ ]:
#mnist epoch 10 rff-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 0.9820 81.25
99 0.3640 81.25
149 0.1924 93.75
199 0.1704 93.75
249 0.2110 93.75
299 0.1443 100.0
349 0.5804 75.0
399 0.1857 93.75
449 0.2395 87.5
499 0.5235 81.25
549 0.2044 93.75
599 0.5589 75.0
649 0.1620 93.75
699 0.3172 93.75
749 0.1744 93.75
799 0.1649 93.75
849 0.4774 87.5
899 0.3549 81.25
949 0.1349 93.75
999 0.1843 93.75
1049 0.1660 87.5
1099 0.3242 87.5
1149 0.2789 87.5
1199 0.1363 93.75
1249 0.2294 87.5
1299 0.2810 87.5
1349 0.0472 100.0
1399 0.2559 93.75
1449 0.0752 100.0
1499 0.2567 93.75
1549 0.0384 100.0
1599 0.0919 100.0
1649 0.0995 100.0
1699 0.1512 100.0
1749 0.0860 100.0
1799 0.1760 87.5
1849 0.1624 87.5
1899 0.2381 93.75
1949 0.1418 93.75
1999 0.0

## FMNIST

In [ ]:
#fminst epoch 1 rff-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 60000 10000
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([0.])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
12800 images encoded. Total time elapse = 26.958014726638794
25600 images encoded. Total time elapse = 54.09557890892029
38400 images encoded. Total time elapse = 81.32328343391418
51200 images encoded. Total time elapse = 108.40847945213318
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49

In [ ]:
#fminst epoch 10 rff-hdc
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-hdc'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Use binary HDC with random fourier features, ignoring gorder, set to 2.
Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 0.9590 68.75
99 0.5808 81.25
149 0.5005 87.5
199 0.5304 87.5
249 0.4107 87.5
299 0.6679 75.0
349 0.1480 100.0
399 0.5236 81.25
449 0.7350 62.5
499 0.3320 87.5
549 0.2477 93.75
599 0.4652 81.25
649 0.2684 93.75
699 0.5048 81.25
749 0.8694 75.0
799 0.3789 81.25
849 0.2121 100.0
899 0.6613 75.0
949 0.2467 93.75
999 0.3966 87.5
1049 1.0813 68.75
1099 0.8678 68.75
1149 0.1634 100.0
1199 0.3428 87.5
1249 0.2436 87.5
1299 0.1887 100.0
1349 0.1117 100.0
1399 0.5783 75.0
1449 0.3533 81.25
1499 0.3452 87.5
1549 0.7766 81.25
1599 0.6449 68.75
1649 0.5125 81.25
1699 0.3682 87.5
1749 0.7950 75.0
1799 0.5008 81.25
1849 0.2837 87.5
1899 0.6861 75.0
1949 0.4638 93.75
1999 0.5054 87

# RFF G($2^1$)-VSA

## ISOLET

In [ ]:
#isolet epoch 1 rffg2vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 2,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 6238 1559
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([0.])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 1.9258 62.5
99 0.5204 93.75
149 0.4092 87.5
199 0.1536 100.0
249 0.2872 87.5
299 0.3097 87.5
349 0.0613 100.0
Start testing on test set

Testing Average loss: 0.0201, Accuracy: 1402/1559 (89.93%)

--- 2.3380937576293945 seconds ---


In [ ]:
#isolet epoch 10 rffg2vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 2,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 1.9179 62.5
99 0.7909 87.5
149 0.1490 93.75
199 0.7417 75.0
249 0.4739 81.25
299 0.5101 87.5
349 0.1667 87.5
Start testing on test set

Testing Average loss: 0.0231, Accuracy: 1394/1559 (89.42%)

--- 2.451706886291504 seconds ---
Epoch: 2
49 0.1609 93.75
99 0.0116 100.0
149 0.0032 100.0
199 0.0029 100.0
249 0.0116 100.0
299 0.0044 100.0
349 0.0050 100.0
Start testing on test set

Testing Average loss: 0.0169, Accuracy: 1434/1559 (91.98%)

--- 1.9702389240264893 seconds ---
Epoch: 3
49 0.0032 100.0
99 0.0005 100.0
149 0.0011 100.0
199 0.0028 100.0
249 0.0006 100.0
299 0.0012 100.0
349 0.0026 100.0
Start testing on test set

Testing Average loss: 0.0155, Accuracy: 1439/1559 (92.30%)

--- 1.9846508502960205 seconds ---
Epoch: 4
49 0.0007 100

## UCIHAR

In [ ]:
#ucihar epoch 1 rffg2vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 2,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 7352 2947
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([0.])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 0.5121 75.0
99 0.0914 100.0
149 0.0829 100.0
199 0.0277 100.0
249 0.0409 100.0
299 0.0792 93.75
349 0.0384 100.0
399 0.0257 100.0
449 0.0586 100.0
Start testing on test set

Testing Average loss: 0.0131, Accuracy: 2730/2947 (92.64%)

--- 3.291971445083618 seconds ---


In [ ]:
#ucihar epoch 10 rffg2vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 2,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 0.2738 93.75
99 0.1169 100.0
149 0.1295 93.75
199 0.0868 100.0
249 0.0485 100.0
299 0.0551 100.0
349 0.0487 100.0
399 0.0140 100.0
449 0.0656 93.75
Start testing on test set

Testing Average loss: 0.0089, Accuracy: 2804/2947 (95.15%)

--- 2.5409297943115234 seconds ---
Epoch: 2
49 0.0054 100.0
99 0.0545 93.75
149 0.0192 100.0
199 0.0201 100.0
249 0.0308 100.0
299 0.0144 100.0
349 0.0298 100.0
399 0.0015 100.0
449 0.0035 100.0
Start testing on test set

Testing Average loss: 0.0093, Accuracy: 2783/2947 (94.44%)

--- 2.466930866241455 seconds ---
Epoch: 3
49 0.0018 100.0
99 0.0002 100.0
149 0.0003 100.0
199 0.0141 100.0
249 0.0005 100.0
299 0.0008 100.0
349 0.0054 100.0
399 0.0116 100.0
449 0.0139 100.0
Start testing on test set

Testing Av

## MNIST

In [ ]:
#mnist epoch 1 rffg2vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 2,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 60000 10000
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([0.])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
12800 images encoded. Total time elapse = 22.91481375694275
25600 images encoded. Total time elapse = 45.82794117927551
38400 images encoded. Total time elapse = 68.59516954421997
51200 images encoded. Total time elapse = 91.45209884643555
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 0.9618 100.0
99 0.5312 75.0
149 0.1918 93.75
199 0.3259 87.5
249 0.2240 9

In [ ]:
#mnist epoch 10 rffg2vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 2,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 1.5304 62.5
99 0.4867 87.5
149 0.1419 93.75
199 0.1182 100.0
249 0.1092 93.75
299 0.2597 93.75
349 0.5081 81.25
399 0.1487 93.75
449 0.3419 87.5
499 0.2892 81.25
549 0.0666 100.0
599 0.4737 81.25
649 0.1359 93.75
699 0.1921 93.75
749 0.1802 93.75
799 0.1127 100.0
849 0.6034 87.5
899 0.4002 75.0
949 0.1544 93.75
999 0.1460 93.75
1049 0.2063 87.5
1099 0.1659 93.75
1149 0.1445 93.75
1199 0.0405 100.0
1249 0.0243 100.0
1299 0.0858 93.75
1349 0.0117 100.0
1399 0.4560 87.5
1449 0.0628 100.0
1499 0.3606 81.25
1549 0.0364 100.0
1599 0.0215 100.0
1649 0.0935 93.75
1699 0.0750 100.0
1749 0.0957 93.75
1799 0.2145 87.5
1849 0.2135 93.75
1899 0.2517 93.75
1949 0.2429 93.75
1999 0.0327 100.0
2049 0.0685 100.0
2099 0.4189 93.75
2149 0.0745 100.0
2199 

## FMNIST

In [ ]:
#fminst epoch 1 rffg2vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 2,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 60000 10000
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([0.])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
12800 images encoded. Total time elapse = 26.832853317260742
25600 images encoded. Total time elapse = 53.6350998878479
38400 images encoded. Total time elapse = 80.54456806182861
51200 images encoded. Total time elapse = 107.44296073913574
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 1.1921 81.25
99 1.1890 62.5
149 1.0346 56.25
199 0.2096 93.75
249 0.2326

In [ ]:
#fminst epoch 10 rffg2vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 2,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 1.0141 75.0
99 0.7540 75.0
149 0.6296 87.5
199 0.5660 68.75
249 0.4189 81.25
299 0.5472 81.25
349 0.1106 100.0
399 0.6538 75.0
449 0.4522 87.5
499 0.2475 93.75
549 0.4516 93.75
599 0.2532 93.75
649 0.1557 100.0
699 0.3438 93.75
749 0.7175 81.25
799 0.3458 87.5
849 0.2399 100.0
899 0.6001 75.0
949 0.2190 93.75
999 0.5107 75.0
1049 0.8740 75.0
1099 0.6294 75.0
1149 0.1861 100.0
1199 0.3055 93.75
1249 0.1911 93.75
1299 0.1205 100.0
1349 0.2276 93.75
1399 0.6187 87.5
1449 0.4791 87.5
1499 0.5584 81.25
1549 0.7981 81.25
1599 0.4997 75.0
1649 0.5629 81.25
1699 0.3080 87.5
1749 0.6065 81.25
1799 0.3636 93.75
1849 0.2220 93.75
1899 0.5691 68.75
1949 0.5324 75.0
1999 0.4261 87.5
2049 0.4685 87.5
2099 0.6243 75.0
2149 0.3425 87.5
2199 0.5475 93.7

# RFF G($2^2$)-VSA

## ISOLET

In [ ]:
#isolet epoch 1 rffg4vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 4,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 6238 1559
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-0.6746,  0.0000,  0.6746])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 2.0968 62.5
99 1.1331 75.0
149 0.5840 81.25
199 0.3061 93.75
249 0.4890 81.25
299 0.5003 75.0
349 0.1824 100.0
Start testing on test set

Testing Average loss: 0.0597, Accuracy: 1342/1559 (86.08%)

--- 2.604689836502075 seconds ---


In [ ]:
#isolet epoch 10 rffg4vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 4,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 1.9322 68.75
99 1.2453 68.75
149 0.1989 100.0
199 0.7047 81.25
249 0.3389 100.0
299 0.5219 87.5
349 0.1699 100.0
Start testing on test set

Testing Average loss: 0.0590, Accuracy: 1355/1559 (86.91%)

--- 2.1330695152282715 seconds ---
Epoch: 2
49 0.0041 100.0
99 0.0106 100.0
149 0.0087 100.0
199 0.0040 100.0
249 0.0053 100.0
299 0.0116 100.0
349 0.0018 100.0
Start testing on test set

Testing Average loss: 0.0491, Accuracy: 1365/1559 (87.56%)

--- 1.9538180828094482 seconds ---
Epoch: 3
49 0.0031 100.0
99 0.0015 100.0
149 0.0028 100.0
199 0.0028 100.0
249 0.0006 100.0
299 0.0019 100.0
349 0.0018 100.0
Start testing on test set

Testing Average loss: 0.0443, Accuracy: 1382/1559 (88.65%)

--- 1.9314517974853516 seconds ---
Epoch: 4
49 0.001

## UCIHAR

In [ ]:
#ucihar epoch 1 rffg4vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 4,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 7352 2947
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-0.6746,  0.0000,  0.6746])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 0.4102 87.5
99 0.0826 100.0
149 0.0697 100.0
199 0.0237 100.0
249 0.0495 100.0
299 0.1174 93.75
349 0.0587 100.0
399 0.0211 100.0
449 0.1339 93.75
Start testing on test set

Testing Average loss: 0.0350, Accuracy: 2671/2947 (90.63%)

--- 3.3710241317749023 seconds ---


In [ ]:
#ucihar epoch 10 rffg4vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 4,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 0.2205 93.75
99 0.0896 100.0
149 0.1381 93.75
199 0.1145 93.75
249 0.0700 93.75
299 0.0570 100.0
349 0.0749 93.75
399 0.0180 100.0
449 0.0863 93.75
Start testing on test set

Testing Average loss: 0.0341, Accuracy: 2733/2947 (92.74%)

--- 2.577092170715332 seconds ---
Epoch: 2
49 0.0062 100.0
99 0.0227 100.0
149 0.0161 100.0
199 0.0069 100.0
249 0.0041 100.0
299 0.0197 100.0
349 0.0089 100.0
399 0.0008 100.0
449 0.0034 100.0
Start testing on test set

Testing Average loss: 0.0189, Accuracy: 2766/2947 (93.86%)

--- 2.490126132965088 seconds ---
Epoch: 3
49 0.0008 100.0
99 0.0004 100.0
149 0.0006 100.0
199 0.0220 100.0
249 0.0008 100.0
299 0.0001 100.0
349 0.0006 100.0
399 0.0018 100.0
449 0.0143 100.0
Start testing on test set

Testing Ave

## MNIST

In [ ]:
#mnist epoch 1 rffg4vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 4,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 60000 10000
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-0.6746,  0.0000,  0.6746])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
12800 images encoded. Total time elapse = 22.603414297103882
25600 images encoded. Total time elapse = 45.186030626297
38400 images encoded. Total time elapse = 67.87335205078125
51200 images encoded. Total time elapse = 90.62863206863403
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 1.3103 93.75
99 0.8486 75.0
149 0.3778 93.75
199 0.

In [ ]:
#mnist epoch 10 rffg4vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 4,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 1.7337 56.25
99 0.7384 81.25
149 0.6409 87.5
199 0.3822 93.75
249 0.1441 100.0
299 0.2714 93.75
349 0.6173 75.0
399 0.4280 81.25
449 0.3399 93.75
499 0.5066 81.25
549 0.2266 87.5
599 0.7567 81.25
649 0.1944 93.75
699 0.2545 93.75
749 0.0634 100.0
799 0.6474 68.75
849 0.3484 87.5
899 0.4407 81.25
949 0.1556 93.75
999 0.1685 93.75
1049 0.1616 93.75
1099 0.3389 93.75
1149 0.4818 87.5
1199 0.0472 100.0
1249 0.0362 100.0
1299 0.6640 87.5
1349 0.2134 87.5
1399 0.3297 87.5
1449 0.0425 100.0
1499 0.1942 100.0
1549 0.0748 93.75
1599 0.1170 93.75
1649 0.3775 87.5
1699 0.0483 100.0
1749 0.2319 93.75
1799 0.1033 100.0
1849 0.1058 93.75
1899 0.2278 93.75
1949 0.2042 87.5
1999 0.0881 100.0
2049 0.1311 93.75
2099 0.4273 75.0
2149 0.0866 93.75
2199 0.0

## FMNIST

In [ ]:
#fminst epoch 1 rffg4vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 4,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 60000 10000
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-0.6746,  0.0000,  0.6746])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
12800 images encoded. Total time elapse = 26.797804832458496
25600 images encoded. Total time elapse = 54.070013761520386
38400 images encoded. Total time elapse = 80.93772220611572
51200 images encoded. Total time elapse = 107.74367713928223
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 1.4405 68.75
99 1.4364 43.75
149 1.2066 56.25
1

In [ ]:
#fminst epoch 10 rffg4vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 4,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 1.2674 87.5
99 1.0163 75.0
149 1.0957 50.0
199 0.7523 68.75
249 0.4957 87.5
299 0.5855 81.25
349 0.5954 81.25
399 0.7489 75.0
449 0.6624 75.0
499 0.4638 81.25
549 0.3337 93.75
599 0.5963 68.75
649 0.1547 100.0
699 0.7419 75.0
749 1.1166 68.75
799 0.6064 75.0
849 0.2643 93.75
899 0.5212 87.5
949 0.0644 100.0
999 0.3830 87.5
1049 0.6343 75.0
1099 0.5966 87.5
1149 0.1737 100.0
1199 0.4365 81.25
1249 0.3015 93.75
1299 0.3126 87.5
1349 0.2497 93.75
1399 0.2784 93.75
1449 0.3525 87.5
1499 0.8569 75.0
1549 0.6186 81.25
1599 0.6763 81.25
1649 0.6395 81.25
1699 0.2543 87.5
1749 0.4843 81.25
1799 0.5559 81.25
1849 0.3420 87.5
1899 0.6890 87.5
1949 0.5187 87.5
1999 0.5140 81.25
2049 0.3469 93.75
2099 0.6222 75.0
2149 0.5069 81.25
2199 0.6446 87.5


# RFF G($2^3$)-VSA

## ISOLET

In [ ]:
#isolet epoch 1 RFF G8VSA
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 8,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 6238 1559
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.1505, -0.6746, -0.3187,  0.0000,  0.3187,  0.6746,  1.1505])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 1.8414 75.0
99 0.8240 81.25
149 0.4975 93.75
199 0.3391 87.5
249 0.3624 87.5
299 0.3582 87.5
349 0.0888 100.0
Start testing on test set

Testing Average loss: 0.0297, Accuracy: 1446/1559 (92.75%)

--- 2.0775809288024902 seconds ---


In [ ]:
#isolet epoch 10 rff8gvsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 8,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 1.8972 62.5
99 1.0855 68.75
149 0.1954 100.0
199 0.7464 75.0
249 0.3968 81.25
299 0.3751 87.5
349 0.1076 100.0
Start testing on test set

Testing Average loss: 0.0286, Accuracy: 1444/1559 (92.62%)

--- 2.0307114124298096 seconds ---
Epoch: 2
49 0.0133 100.0
99 0.0271 100.0
149 0.0087 100.0
199 0.0097 100.0
249 0.0233 100.0
299 0.0175 100.0
349 0.0047 100.0
Start testing on test set

Testing Average loss: 0.0176, Accuracy: 1464/1559 (93.91%)

--- 1.9907522201538086 seconds ---
Epoch: 3
49 0.0073 100.0
99 0.0052 100.0
149 0.0054 100.0
199 0.0100 100.0
249 0.0032 100.0
299 0.0034 100.0
349 0.0043 100.0
Start testing on test set

Testing Average loss: 0.0147, Accuracy: 1470/1559 (94.29%)

--- 2.6011674404144287 seconds ---
Epoch: 4
49 0.0010 

## UCIHAR

In [ ]:
#ucihar epoch 1 rffg8vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 8,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 7352 2947
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.1505, -0.6746, -0.3187,  0.0000,  0.3187,  0.6746,  1.1505])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 0.6040 81.25
99 0.1575 87.5
149 0.1115 100.0
199 0.0493 100.0
249 0.1137 93.75
299 0.0518 100.0
349 0.0896 93.75
399 0.0637 100.0
449 0.1015 93.75
Start testing on test set

Testing Average loss: 0.0136, Accuracy: 2762/2947 (93.72%)

--- 2.8710765838623047 s

In [ ]:
#ucihar epoch 10 rffg8vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 8,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 0.3539 93.75
99 0.2014 100.0
149 0.1689 100.0
199 0.1881 93.75
249 0.0837 93.75
299 0.0740 100.0
349 0.0995 93.75
399 0.0474 100.0
449 0.0732 93.75
Start testing on test set

Testing Average loss: 0.0130, Accuracy: 2768/2947 (93.93%)

--- 2.4904892444610596 seconds ---
Epoch: 2
49 0.0066 100.0
99 0.1260 93.75
149 0.0055 100.0
199 0.0907 93.75
249 0.0157 100.0
299 0.0167 100.0
349 0.0130 100.0
399 0.0025 100.0
449 0.0046 100.0
Start testing on test set

Testing Average loss: 0.0090, Accuracy: 2789/2947 (94.64%)

--- 2.4219865798950195 seconds ---
Epoch: 3
49 0.0052 100.0
99 0.0016 100.0
149 0.0032 100.0
199 0.0104 100.0
249 0.0010 100.0
299 0.0031 100.0
349 0.0222 100.0
399 0.0034 100.0
449 0.0062 100.0
Start testing on test set

Testing A

## MNIST

In [ ]:
#mnist epoch 1 rffg8vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 8,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 60000 10000
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.1505, -0.6746, -0.3187,  0.0000,  0.3187,  0.6746,  1.1505])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
12800 images encoded. Total time elapse = 23.032389163970947
25600 images encoded. Total time elapse = 46.168821811676025
38400 images encoded. Total time elapse = 69.22472286224365
51200 images encoded. Total time elapse = 92.15534234046936
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 0.9797 100.0

In [ ]:
#mnist epoch 10 rffg8vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 8,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 1.5183 62.5
99 0.5644 81.25
149 0.2876 93.75
199 0.2138 100.0
249 0.1203 93.75
299 0.3441 87.5
349 0.4103 81.25
399 0.2338 93.75
449 0.1695 93.75
499 0.2503 87.5
549 0.2024 93.75
599 0.5539 81.25
649 0.0518 100.0
699 0.1080 93.75
749 0.1177 93.75
799 0.0948 93.75
849 0.4120 87.5
899 0.1409 93.75
949 0.0621 100.0
999 0.0705 100.0
1049 0.0661 100.0
1099 0.1561 93.75
1149 0.0307 100.0
1199 0.1392 93.75
1249 0.0372 100.0
1299 0.1060 93.75
1349 0.0049 100.0
1399 0.1702 93.75
1449 0.0126 100.0
1499 0.0854 100.0
1549 0.0029 100.0
1599 0.0355 100.0
1649 0.1042 93.75
1699 0.0269 100.0
1749 0.0570 100.0
1799 0.1607 87.5
1849 0.0530 100.0
1899 0.0902 100.0
1949 0.0568 100.0
1999 0.0211 100.0
2049 0.0168 100.0
2099 0.3495 93.75
2149 0.0634 100.0
21

## FMNIST

In [ ]:
#fminst epoch 1 rffg8vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 8,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 60000 10000
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.1505, -0.6746, -0.3187,  0.0000,  0.3187,  0.6746,  1.1505])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
12800 images encoded. Total time elapse = 27.11051082611084
25600 images encoded. Total time elapse = 54.34337568283081
38400 images encoded. Total time elapse = 81.67120671272278
51200 images encoded. Total time elapse = 108.89743971824646
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 1.1395 81.25


In [ ]:
#fminst epoch 10 rffg8vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 8,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 0.9050 87.5
99 0.8003 68.75
149 0.7982 68.75
199 0.6604 81.25
249 0.3492 93.75
299 0.5100 87.5
349 0.2224 93.75
399 0.4002 87.5
449 0.3760 93.75
499 0.2147 93.75
549 0.4227 87.5
599 0.3542 81.25
649 0.1335 100.0
699 0.3603 87.5
749 0.8115 75.0
799 0.3730 87.5
849 0.0820 100.0
899 0.3445 87.5
949 0.0761 93.75
999 0.3287 87.5
1049 0.9485 68.75
1099 0.5422 81.25
1149 0.2502 93.75
1199 0.4544 81.25
1249 0.1526 93.75
1299 0.1383 100.0
1349 0.1398 93.75
1399 0.2118 93.75
1449 0.2858 93.75
1499 0.4725 75.0
1549 0.6279 87.5
1599 0.6112 87.5
1649 0.2713 93.75
1699 0.3447 81.25
1749 0.2502 87.5
1799 0.3153 93.75
1849 0.3553 87.5
1899 0.5517 81.25
1949 0.5069 81.25
1999 0.2369 93.75
2049 0.3452 87.5
2099 0.5783 68.75
2149 0.2108 93.75
2199 0.4360 

# RFF G($2^4$)-VSA

## ISOLET

In [ ]:
#isolet epoch 1 rffg16vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 6238 1559
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.5343, -1.1505, -0.8872, -0.6746, -0.4888, -0.3187, -0.1573,  0.0000,
         0.1573,  0.3187,  0.4888,  0.6746,  0.8872,  1.1505,  1.5343])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 2.0995 68.75
99 0.9491 81.25
149 0.6568 81.25
199 0.3804 87.5
249 0.4022 81.25
299 0.4028 87.5
349 0.1018 100.0
Start testing on test set

Testing Average loss: 0.0169, Accuracy:

In [ ]:
#isolet epoch 10 rffg16vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 2.1540 50.0
99 1.1984 75.0
149 0.2346 93.75
199 0.8011 75.0
249 0.3979 81.25
299 0.4434 87.5
349 0.1179 100.0
Start testing on test set

Testing Average loss: 0.0172, Accuracy: 1468/1559 (94.16%)

--- 2.463609218597412 seconds ---
Epoch: 2
49 0.0426 100.0
99 0.0932 100.0
149 0.0388 100.0
199 0.0176 100.0
249 0.0797 100.0
299 0.0440 100.0
349 0.0234 100.0
Start testing on test set

Testing Average loss: 0.0111, Accuracy: 1479/1559 (94.87%)

--- 1.9738655090332031 seconds ---
Epoch: 3
49 0.0171 100.0
99 0.0166 100.0
149 0.0161 100.0
199 0.0541 100.0
249 0.0084 100.0
299 0.0108 100.0
349 0.0121 100.0
Start testing on test set

Testing Average loss: 0.0097, Accuracy: 1482/1559 (95.06%)

--- 2.017972230911255 seconds ---
Epoch: 4
49 0.0033 100

## UCIHAR

In [ ]:
#ucihar epoch 1 rffg16vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 7352 2947
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.5343, -1.1505, -0.8872, -0.6746, -0.4888, -0.3187, -0.1573,  0.0000,
         0.1573,  0.3187,  0.4888,  0.6746,  0.8872,  1.1505,  1.5343])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 0.8452 62.5
99 0.2788 87.5
149 0.1851 100.0
199 0.1250 100.0
249 0.1513 93.75
299 0.0681 100.0
349 0.1164 93.75
399 0.1283 100.0
449 0.1121 93.75
Start testing on test set

Testi

In [ ]:
#ucihar epoch 10 rffg16vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 0.5797 87.5
99 0.4076 93.75
149 0.2845 87.5
199 0.2783 87.5
249 0.1712 93.75
299 0.1226 93.75
349 0.1966 87.5
399 0.0676 100.0
449 0.0829 93.75
Start testing on test set

Testing Average loss: 0.0105, Accuracy: 2787/2947 (94.57%)

--- 2.4394707679748535 seconds ---
Epoch: 2
49 0.0159 100.0
99 0.1159 93.75
149 0.0172 100.0
199 0.1342 100.0
249 0.0511 100.0
299 0.0260 100.0
349 0.0154 100.0
399 0.0097 100.0
449 0.0115 100.0
Start testing on test set

Testing Average loss: 0.0078, Accuracy: 2818/2947 (95.62%)

--- 3.181349277496338 seconds ---
Epoch: 3
49 0.0121 100.0
99 0.0102 100.0
149 0.0032 100.0
199 0.0256 100.0
249 0.0035 100.0
299 0.0095 100.0
349 0.1119 93.75
399 0.0098 100.0
449 0.0257 100.0
Start testing on test set

Testing Averag

## MNIST

In [ ]:
#mnist epoch 1 rffg16vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 60000 10000
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.5343, -1.1505, -0.8872, -0.6746, -0.4888, -0.3187, -0.1573,  0.0000,
         0.1573,  0.3187,  0.4888,  0.6746,  0.8872,  1.1505,  1.5343])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
12800 images encoded. Total time elapse = 23.111278772354126
25600 images encoded. Total time elapse = 46.25865411758423
38400 images encoded. Total time elapse = 69.27807807922363
51200 images encoded. Total time elapse = 92.28337907791138
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6

In [ ]:
#mnist epoch 10 rffg16vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 1.5492 68.75
99 0.5797 87.5
149 0.2853 93.75
199 0.1788 100.0
249 0.1036 100.0
299 0.3539 81.25
349 0.3697 75.0
399 0.2319 93.75
449 0.2126 87.5
499 0.3083 81.25
549 0.1738 87.5
599 0.4366 81.25
649 0.0674 100.0
699 0.0882 93.75
749 0.0677 100.0
799 0.1423 100.0
849 0.4121 87.5
899 0.1597 93.75
949 0.0490 100.0
999 0.1582 93.75
1049 0.0594 100.0
1099 0.1573 93.75
1149 0.0354 100.0
1199 0.0602 100.0
1249 0.0620 100.0
1299 0.0447 100.0
1349 0.0101 100.0
1399 0.1730 93.75
1449 0.0313 100.0
1499 0.1358 93.75
1549 0.0145 100.0
1599 0.0576 100.0
1649 0.0722 93.75
1699 0.0700 93.75
1749 0.0435 100.0
1799 0.1330 93.75
1849 0.1034 100.0
1899 0.1491 93.75
1949 0.1142 93.75
1999 0.0782 100.0
2049 0.0054 100.0
2099 0.2290 93.75
2149 0.1540 93.75
21

## FMNIST

In [ ]:
#fminst epoch 1 rffg16vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 60000 10000
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.5343, -1.1505, -0.8872, -0.6746, -0.4888, -0.3187, -0.1573,  0.0000,
         0.1573,  0.3187,  0.4888,  0.6746,  0.8872,  1.1505,  1.5343])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
12800 images encoded. Total time elapse = 27.142229557037354
25600 images encoded. Total time elapse = 54.38840985298157
38400 images encoded. Total time elapse = 81.68396663665771
51200 images encoded. Total time elapse = 108.8210518360138
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6

In [ ]:
#fminst epoch 10 rffg16vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 16,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 0.8041 93.75
99 0.7669 75.0
149 0.7923 62.5
199 0.5341 81.25
249 0.4300 87.5
299 0.4295 81.25
349 0.2961 81.25
399 0.3494 87.5
449 0.3736 87.5
499 0.2432 93.75
549 0.2971 93.75
599 0.2208 87.5
649 0.1537 93.75
699 0.3024 93.75
749 0.6402 75.0
799 0.3485 87.5
849 0.0934 100.0
899 0.2670 87.5
949 0.0810 100.0
999 0.3599 81.25
1049 0.6758 81.25
1099 0.4542 87.5
1149 0.1788 93.75
1199 0.1975 93.75
1249 0.1341 93.75
1299 0.1457 93.75
1349 0.2346 87.5
1399 0.1789 87.5
1449 0.1848 93.75
1499 0.3786 75.0
1549 0.5201 87.5
1599 0.4463 81.25
1649 0.2644 81.25
1699 0.2733 87.5
1749 0.1503 93.75
1799 0.2226 93.75
1849 0.2084 93.75
1899 0.4137 87.5
1949 0.3125 87.5
1999 0.2820 87.5
2049 0.3287 87.5
2099 0.4737 75.0
2149 0.0928 100.0
2199 0.3912 93.75

# G($2^5$)-VSA

## ISOLET

In [ ]:
#isolet epoch 1 rffg32vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 32,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 6238 1559
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.8629, -1.5343, -1.3182, -1.1505, -1.0101, -0.8872, -0.7765, -0.6746,
        -0.5792, -0.4888, -0.4023, -0.3187, -0.2372, -0.1573, -0.0784,  0.0000,
         0.0784,  0.1573,  0.2372,  0.3187,  0.4023,  0.4888,  0.5792,  0.6746,
         0.7765,  0.8872,  1.0101,  1.1505,  1.3182,  1.5343,  1.8629])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 2.5880 50.0
99 1.6

In [ ]:
#isolet epoch 10 rffg32vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 32,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'isolet',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([6238, 10000]) and test data torch.Size([1559, 10000])
Epoch: 1
49 2.6985 31.25
99 1.8406 56.25
149 0.5459 100.0
199 0.9736 68.75
249 0.5405 81.25
299 0.5080 87.5
349 0.2111 100.0
Start testing on test set

Testing Average loss: 0.0212, Accuracy: 1452/1559 (93.14%)

--- 2.117562770843506 seconds ---
Epoch: 2
49 0.1760 93.75
99 0.2331 93.75
149 0.1206 100.0
199 0.0513 100.0
249 0.2029 93.75
299 0.0600 100.0
349 0.1174 93.75
Start testing on test set

Testing Average loss: 0.0129, Accuracy: 1483/1559 (95.13%)

--- 1.971390724182129 seconds ---
Epoch: 3
49 0.0518 100.0
99 0.0393 100.0
149 0.0412 100.0
199 0.2849 87.5
249 0.0486 100.0
299 0.0579 100.0
349 0.0409 100.0
Start testing on test set

Testing Average loss: 0.0117, Accuracy: 1465/1559 (93.97%)

--- 2.0153322219848633 seconds ---
Epoch: 4
49 0.0076 1

## UCIHAR

In [ ]:
#ucihar epoch 1 rffg32vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 32,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 7352 2947
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.8629, -1.5343, -1.3182, -1.1505, -1.0101, -0.8872, -0.7765, -0.6746,
        -0.5792, -0.4888, -0.4023, -0.3187, -0.2372, -0.1573, -0.0784,  0.0000,
         0.0784,  0.1573,  0.2372,  0.3187,  0.4023,  0.4888,  0.5792,  0.6746,
         0.7765,  0.8872,  1.0101,  1.1505,  1.3182,  1.5343,  1.8629])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding and saving
Optimizing class representatives for 1 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 1.0779 37.5
99 0.5

In [ ]:
#ucihar epoch 10 rffg32vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 32,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'ucihar',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([7352, 10000]) and test data torch.Size([2947, 10000])
Epoch: 1
49 0.8994 75.0
99 0.7113 68.75
149 0.4762 75.0
199 0.4215 81.25
249 0.3189 81.25
299 0.1786 93.75
349 0.2867 87.5
399 0.1463 100.0
449 0.1418 93.75
Start testing on test set

Testing Average loss: 0.0135, Accuracy: 2729/2947 (92.60%)

--- 2.8200371265411377 seconds ---
Epoch: 2
49 0.0385 100.0
99 0.1366 93.75
149 0.0526 100.0
199 0.2466 75.0
249 0.1228 100.0
299 0.0685 100.0
349 0.0413 100.0
399 0.0222 100.0
449 0.0215 100.0
Start testing on test set

Testing Average loss: 0.0089, Accuracy: 2812/2947 (95.42%)

--- 3.1088883876800537 seconds ---
Epoch: 3
49 0.0480 100.0
99 0.0493 100.0
149 0.0146 100.0
199 0.0875 93.75
249 0.0265 100.0
299 0.0223 100.0
349 0.0974 100.0
399 0.0353 100.0
449 0.0721 100.0
Start testing on test set

Testing Avera

## MNIST

In [ ]:
#mnist epoch 1 rffg32vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 32,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 60000 10000
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.8629, -1.5343, -1.3182, -1.1505, -1.0101, -0.8872, -0.7765, -0.6746,
        -0.5792, -0.4888, -0.4023, -0.3187, -0.2372, -0.1573, -0.0784,  0.0000,
         0.0784,  0.1573,  0.2372,  0.3187,  0.4023,  0.4888,  0.5792,  0.6746,
         0.7765,  0.8872,  1.0101,  1.1505,  1.3182,  1.5343,  1.8629])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
12800 images encoded. Total time elapse = 22.589444875717163
25600 images encoded. Total time elapse = 45.20040726661682
38400 images encoded. Total time elapse = 69.20489263534546
51200 images encoded. Total time elapse = 92.57166004180908
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding an

In [ ]:
#mnist epoch 10 rffg32vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 32,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'mnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 1.8620 43.75
99 0.7994 81.25
149 0.4383 93.75
199 0.3310 100.0
249 0.1780 100.0
299 0.4666 81.25
349 0.4385 75.0
399 0.3052 93.75
449 0.3609 81.25
499 0.2869 87.5
549 0.1467 100.0
599 0.6299 75.0
649 0.0974 100.0
699 0.1343 100.0
749 0.1033 93.75
799 0.2795 87.5
849 0.4467 87.5
899 0.2145 93.75
949 0.1088 100.0
999 0.2838 93.75
1049 0.0475 100.0
1099 0.1929 93.75
1149 0.0500 100.0
1199 0.0797 100.0
1249 0.1630 93.75
1299 0.1221 93.75
1349 0.0265 100.0
1399 0.2546 87.5
1449 0.0282 100.0
1499 0.1408 87.5
1549 0.0169 100.0
1599 0.0303 100.0
1649 0.1208 93.75
1699 0.1225 93.75
1749 0.0390 100.0
1799 0.1322 93.75
1849 0.0826 93.75
1899 0.1964 93.75
1949 0.1441 93.75
1999 0.0732 100.0
2049 0.0133 100.0
2099 0.4554 93.75
2149 0.2924 93.75
2199

## FMNIST

In [ ]:
#fminst epoch 1 rffg32vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 1,            # Epochs of training
            'gorder': 32,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': False,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args

args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encode the dataset into hypervectors and save
# of channels of data 1
# of training samples and test samples 60000 10000
Encoding with random fourier features encoder.
the threshold to discretize fourier features to group elements tensor([-1.8629, -1.5343, -1.3182, -1.1505, -1.0101, -0.8872, -0.7765, -0.6746,
        -0.5792, -0.4888, -0.4023, -0.3187, -0.2372, -0.1573, -0.0784,  0.0000,
         0.0784,  0.1573,  0.2372,  0.3187,  0.4023,  0.4888,  0.5792,  0.6746,
         0.7765,  0.8872,  1.0101,  1.1505,  1.3182,  1.5343,  1.8629])
Encoded pixels to hypervectors with size:  torch.Size([256, 10000])
Encoding training data...
Start encoding data
12800 images encoded. Total time elapse = 26.83484959602356
25600 images encoded. Total time elapse = 54.309898138046265
38400 images encoded. Total time elapse = 82.32806849479675
51200 images encoded. Total time elapse = 111.32183146476746
Finish encoding data
Encoding test data...
Start encoding data
Finish encoding data
Finish encoding a

In [ ]:
#fminst epoch 10 rffg32vsa
class ArgumentParser:
    def __init__(self):
        self.args = {
            'lr': 0.01,            # Learning rate for optimizing class representative
            'gamma': 0.3,          # Kernel parameter for computing covariance
            'epoch': 10,            # Epochs of training
            'gorder': 32,           # Order of the cyclic group required for G-VSA (8 ou 16)
            'dim': 10000,          # Dimension of hypervectors c le grand D
            'seed': 43,            # Random seed for reproducing results
            'resume': True,       # Resume from existing encoded hypervectors
            'data_dir': './encoded_data', # Directory to save encoded data (hypervectors)
            'dataset': 'fmnist',    # Dataset choice choices=['mnist', 'fmnist', 'cifar', 'isolet', 'ucihar']
            'raw_data_dir': './dataset',   # Raw data directory
            'model': 'rff-gvsa'    # Model type choices=['rff-hdc', 'linear-hdc', 'rff-gvsa']
        }
        self.seed = self.args['seed']
        self.lr = self.args['lr']
        self.gamma = self.args['gamma']
        self.epoch = self.args['epoch']
        self.gorder = self.args['gorder']
        self.dim = self.args['dim']
        self.resume = self.args['resume']
        self.data_dir = self.args['data_dir']
        self.dataset = self.args['dataset']
        self.raw_data_dir = self.args['raw_data_dir']
        self.model = self.args['model']


    def get_args(self):
        return self.args
args = ArgumentParser()
torch.manual_seed(args.seed)
np.random.seed(args.seed)
if 'hdc' in args.model:
    args.gorder = 2
    print("Use binary HDC with random fourier features, ignoring gorder, set to 2.")
args.data_dir = f'{args.data_dir}/{args.dataset}_{args.model}_order{args.gorder}_gamma{args.gamma}_dim{args.dim}'
try:
    os.makedirs(args.data_dir)
except FileExistsError:
    print('Encoded data folder already exists')
if not args.resume:
    print('Encode the dataset into hypervectors and save')
    encode_and_save(args)
    print('Finish encoding and saving')
print(f'Optimizing class representatives for {args.epoch} epochs')
train(args)

Encoded data folder already exists
Optimizing class representatives for 10 epochs
Loading encoded training data...
Loading encoded test data...
Size of encoded training data torch.Size([60000, 10000]) and test data torch.Size([10000, 10000])
Epoch: 1
49 1.1208 81.25
99 0.8713 62.5
149 0.8456 62.5
199 0.6049 81.25
249 0.4636 87.5
299 0.4735 81.25
349 0.3627 87.5
399 0.4051 87.5
449 0.4054 93.75
499 0.1770 93.75
549 0.3956 93.75
599 0.3039 81.25
649 0.1670 93.75
699 0.2109 100.0
749 0.5622 75.0
799 0.4233 87.5
849 0.0832 100.0
899 0.3651 81.25
949 0.1344 93.75
999 0.4265 87.5
1049 0.5765 75.0
1099 0.5031 87.5
1149 0.1599 87.5
1199 0.2276 93.75
1249 0.1941 87.5
1299 0.1275 100.0
1349 0.1697 87.5
1399 0.2396 93.75
1449 0.2182 87.5
1499 0.3913 75.0
1549 0.5966 87.5
1599 0.4244 81.25
1649 0.3110 81.25
1699 0.3865 81.25
1749 0.1692 93.75
1799 0.2767 93.75
1849 0.2255 93.75
1899 0.4583 75.0
1949 0.2945 87.5
1999 0.3203 87.5
2049 0.4123 87.5
2099 0.4127 87.5
2149 0.1529 100.0
2199 0.4361 93.75
